In [3]:
# import necessary modules
import keras
import keras.backend as K
from keras.layers import Dense,Conv2D,Input,MaxPooling2D,Flatten
from keras.models import Model
from keras.datasets import fashion_mnist
from keras.callbacks import ModelCheckpoint

In [4]:
# setup parameters
batch_size = 128
nb_class = 10
nb_epochs = 10
img_h,img_w = 28,28

In [5]:
# download and prepare dataset for training and validation
def get_dataset():
    """
    Return processed fashion-mnist dataset
    """
    # load mnist dataset
    (x_train,y_train),(x_test,y_test) = fashion_mnist.load_data()

    # test and train datasets
    print("Nb Train:",x_train.shape[0],"Nb test:", x_test.shape[0])
    x_train = x_train.reshape(x_train.shape[0],img_h,img_w,1)
    x_test = x_test.reshape(x_test.shape[0],img_h,img_w,1)
    in_shape = (img_h,img_w,1)

    # normalize inputs
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255.0
    x_test /= 255.0

    # convert to one hot vectors
    y_train = keras.utils.to_categorical(y_train,nb_class)
    y_test = keras.utils.to_categorical(y_test,nb_class)
    
    return x_train,x_test,y_train,y_test

x_train,y_train,x_test,y_test = get_dataset()

4422102/4422102 [==============================] - 2s 0us/step
Nb Train: 60000 Nb test: 10000


##### **Build Model**

In [6]:
def conv3x3(input_x,nb_filters):
    """
    Wrapper around convolution layer
    Inputs:
        input_x:input layer/tensor
        nb_filter:Number of filters for convolution
    """
    return Conv2D(nb_filters,kernel_size=(3,3),use_bias=False,
                  activation='relu',padding='same')(input_x)


In [7]:
def create_model(img_h=28,img_w=28):
    """
    Creates a CNN model for training
    Inputs:
        img_h:input image height
        img_w:imput image width
    Returns:
        model structure
    """
    inputs = Input(shape=(img_h,img_w,1))

    x = conv3x3(inputs,32)
    x = conv3x3(x,32)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = conv3x3(x,64)
    x = conv3x3(x,64)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Flatten()(x)
    x = Dense(128,activation='relu')(x)
    preds = Dense(nb_class,activation='softmax')(x)
    model = Model(inputs=inputs,outputs=preds)
    print(model.summary())
    return model
    

In [8]:
# instatiate model
model = create_model()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 32)        288       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        9216      
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 32)        0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 64)        18432     
                                                                 
 conv2d_3 (Conv2D)           (None, 14, 14, 64)        36864     
                                                             

In [9]:
# set up optimizer, loss function and metrics
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer = keras.optimizers.Adam(),
              metrics = ['accuracy'])

In [10]:
# save model after every epoch
callback = ModelCheckpoint('mnist_cnn.h5')

In [11]:
# begin training model
model.fit(x_train,y_train,
          batch_size = batch_size,
          epochs = nb_epochs,
          verbose = 1,
          validation_data = (x_test,y_test),
          callbacks = [callback])

2023-10-09 14:57:26.038693: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.
2023-10-09 14:57:26.262517: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 31360000 exceeds 10% of free system memory.


ValueError: Data cardinality is ambiguous:
  x sizes: 60000
  y sizes: 10000
Make sure all arrays contain the same number of samples.